In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.contrib.framework.python.ops import audio_ops
from tensorflow.python.ops import io_ops

In [13]:
def filename(path):
	return os.path.splitext(os.path.split(path)[1])[0]

def prepare_model_settings(sample_rate, clip_duration_ms,
                           window_size_ms, window_stride_ms,
                           dct_coefficient_count):
    """Calculates common settings needed for all models.

  Args:
    sample_rate: Number of audio samples per second.
    clip_duration_ms: Length of each audio clip to be analyzed.
    window_size_ms: Duration of frequency analysis window.
    window_stride_ms: How far to move in time between frequency windows.
    dct_coefficient_count: Number of frequency bins to use for analysis.

  Returns:
    Dictionary containing common settings.
  """
    desired_samples = int(sample_rate * clip_duration_ms / 1000)
    window_size_samples = int(sample_rate * window_size_ms / 1000)
    window_stride_samples = int(sample_rate * window_stride_ms / 1000)
    length_minus_window = (desired_samples - window_size_samples)
    if length_minus_window < 0:
        spectrogram_length = 0
    else:
        spectrogram_length = 1 + int(length_minus_window / window_stride_samples)
        fingerprint_size = dct_coefficient_count * spectrogram_length
    return {
              'desired_samples': desired_samples,
              'window_size_samples': window_size_samples,
              'window_stride_samples': window_stride_samples,
              'spectrogram_length': spectrogram_length,
              'dct_coefficient_count': dct_coefficient_count,
              'fingerprint_size': fingerprint_size,
              'sample_rate': sample_rate,
            }

def get_mfcc_graph(model_settings):
	g = tf.Graph()
	with g.as_default():
		input_file_placeholder = tf.compat.v1.placeholder(tf.string, [], name='wav_filename')
		wav_loader = io_ops.read_file(input_file_placeholder)
		wav_decoder = audio_ops.decode_wav(wav_loader, desired_channels=1, desired_samples=model_settings['desired_samples'])
		# Run the spectrogram and MFCC ops to get a 2D 'fingerprint' of the audio.
		spectrograms_power = audio_ops.audio_spectrogram(wav_decoder.audio,
												  window_size=model_settings['window_size_samples'],
												  stride=model_settings['window_stride_samples'],
												  magnitude_squared=True)
		USE_POWER=True
		if USE_POWER:
				# Warp the linear scale spectrograms into the mel-scale.
				num_spectrogram_bins = spectrograms_power.shape[-1].value
				lower_edge_hertz, upper_edge_hertz, num_mel_bins = 20.0, 4000.0, 40
				linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
					num_mel_bins, num_spectrogram_bins, 16000.0, lower_edge_hertz,
					upper_edge_hertz)
				mel_spectrograms = tf.tensordot(
					spectrograms_power, linear_to_mel_weight_matrix, 1)
				mel_spectrograms.set_shape(spectrograms_power.shape[:-1].concatenate(
					linear_to_mel_weight_matrix.shape[-1:]))

				# Compute a stabilized log to get log-magnitude mel-scale spectrograms.
				log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

				# Compute MFCCs from log_mel_spectrograms and take the first NDCT.
				mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
					log_mel_spectrograms)[..., :model_settings['dct_coefficient_count']]
				#output = tf.expand_dims(mfccs, axis=0)
				output = mfccs
		else:
			output = contrib_audio.mfcc(
				 spectrograms_power,
				 wav_decoder.sample_rate,
				 dct_coefficient_count=model_settings['dct_coefficient_count'])
	return g, input_file_placeholder, output, wav_decoder.audio


In [14]:
sample_rate = 16000
clip_duration_ms = 1000
dct_coefficient_count = 10
window_size_ms = 40
window_stride_ms = 20
input_file = "../out_file.wav"
NNTOOL_INPUT_SCALE = 1.9372712
DCT_NORMALIZATION = np.sqrt(1/(dct_coefficient_count))*0.5
FACTOR_SCALE = NNTOOL_INPUT_SCALE*DCT_NORMALIZATION

model_settings = prepare_model_settings(
    	  sample_rate, clip_duration_ms, window_size_ms,
	  	  window_stride_ms, dct_coefficient_count)


print(input_file)
g, input_file_placeholder, tf_mfccs, signal = get_mfcc_graph(model_settings)
with tf.compat.v1.Session(graph=g) as sess:
    tf.initialize_all_variables().run()
    tf_mfccs, signal = sess.run([tf_mfccs, signal], feed_dict={input_file_placeholder: input_file})
    
tf_mfccs_int8 = np.floor(tf_mfccs.astype(np.float32) / NNTOOL_INPUT_SCALE + 0.5).astype(np.int8) # Scale to int8
print(tf_mfccs_int8.shape)
print(FACTOR_SCALE)
FACTOR_SCALEN = 9
print(FACTOR_SCALE * 2**(FACTOR_SCALEN))
print(round(FACTOR_SCALE * 2**(FACTOR_SCALEN)), FACTOR_SCALEN)

../out_file.wav
(1, 49, 10)
0.30630947187237945
156.83044959865828
157 9


In [16]:
#model_path = "model/KWS_ds_cnn_s_quant.tflite"
model_path = "../model/KWS_ds_cnn_m_quant_new.tflite"
#model_path = "model/KWS_ds_cnn_l_quant.tflite"

interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details: \n", input_details)
print("Output Details: \n", output_details)
scale, zero_point = input_details[0]['quantization']

input_array = np.array(np.floor(tf_mfccs / scale + 0.5) + zero_point).astype(np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_array.reshape(input_details[0]['shape']))
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print(output)
print("Predicted: ", np.argmax(output))

Input Details: 
 [{'name': 'Reshape_1', 'index': 31, 'shape': array([ 1, 49, 10,  1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.937271237373352, 128)}]
Output Details: 
 [{'name': 'labels_softmax', 'index': 32, 'shape': array([ 1, 12], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0)}]
[[ 0 19 25 44  6 25 33 19  6  8 11 58]]
Predicted:  11


# Network input

In [6]:
tf_mfccs_int8

array([[[ -12,    3,    1,    1,    1,    1,    0,    0,    0,    0],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    0,    1],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    1,    1],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    1,    0],
        [ -20,    2,    2,    1,    1,    1,    1,    1,    1,    1],
        [ -20,    3,    1,    1,    1,    1,    1,    1,    1,    1],
        [ -20,    2,    2,    1,    1,    1,    1,    1,    0,    1],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    0,    0],
        [ -20,    2,    1,    2,    1,    1,    1,    1,    1,    1],
        [ -20,    2,    1,    1,    1,    2,    1,    1,    1,    1],
        [ -21,    2,    1,    2,    1,    1,    1,    1,    1,    1],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    1,    1],
        [ -20,    2,    1,    1,    1,    1,    1,    1,    0,    0],
        [ -20,    2,    1,    2,    1,    1,    1,    1,    1,    1],
        [ -20,    2,

In [7]:
input_prescale_gap = np.array([
[-821, 46, -17, 2, -5, 0, -4, -15, -15, -11, -3, 0, -8, -10, -8, -8, -9, -5, -6, 2, 1, -10, -12, -7, 4, -13, -12, 1, 2, -2, -8, 1, 0, -1, 1, 2, 2, -2, 2, 0, ],
[-821, 43, -36, -12, -25, -16, -1, -14, -10, -15, 1, -4, -12, -3, 3, 2, -5, 3, 2, 3, 3, -5, -2, 3, 1, -8, -11, -5, -4, 1, -2, -1, -3, 1, -2, 0, 0, -1, -7, -11, ],
[-812, 45, -29, -8, -39, -29, -1, -20, -14, -7, -9, -3, -8, -7, -6, 1, -11, -3, 5, -5, 2, -13, -4, -2, -3, -15, -11, 4, 2, -3, -3, -7, -7, -6, -3, 3, 5, 1, -2, 0, ],
[-819, 22, -8, -22, -2, -26, 3, -16, -36, 7, -18, 2, -12, -7, -19, 5, -9, 6, 18, -6, 3, -6, -7, -9, -5, -1, -5, 1, -3, -2, -7, -6, 0, -1, 1, 0, 0, 3, 1, -8, ],
[-706, -12, 19, -49, 18, -45, -15, -6, -32, 13, -32, 6, -16, -4, -15, -16, -19, -8, -8, -16, -12, -9, -17, -4, -5, -10, 4, -4, 3, -6, -2, -2, -1, -3, -7, 1, 2, 1, 0, -6, ],
[-714, -8, 24, -39, 11, -27, -24, -11, -36, 10, -44, 2, -28, -7, -18, -14, -12, -14, 1, -23, -20, -17, -15, -3, -4, -10, -5, 3, 7, 0, -5, -14, -14, -6, 0, 4, 1, 0, 4, -5, ],
[-791, 8, 15, -10, -6, -14, -8, -20, -21, 6, -26, 5, -15, 2, -13, -10, -19, -17, -10, -12, -2, -7, -11, -9, 1, -10, -7, 9, 10, -1, -3, 0, -8, 0, 2, 1, 5, 3, 9, 1, ],
[-778, 57, 8, -4, -14, -12, -11, -13, -22, 7, -10, 5, -17, 1, -12, -9, -8, -13, 1, -7, -5, -8, -9, 10, 3, -15, 8, 0, 6, -1, -3, 0, -4, 0, -2, 1, -1, 1, 1, -1, ],
[-790, 43, -3, -1, -26, -22, -6, -17, -15, -4, -14, -21, -25, -5, 8, 5, -11, -10, 1, -9, -12, -3, -3, -5, -1, 1, 3, -1, 7, 0, 0, 4, -8, 1, 2, 1, 1, 7, 6, 1, ],
[-803, 46, -11, 8, -29, -22, -12, -15, -14, 0, -9, -22, -17, -9, 1, 4, -3, -4, 4, -6, -4, 6, -7, 9, -6, -11, -5, 4, 1, 5, 8, -1, -6, -5, 1, 5, 13, 6, -3, 0, ],
[-813, 51, 0, 17, -23, -26, -9, -14, -17, 0, -7, -5, -20, -6, -3, -20, -9, -16, -5, 3, -5, -3, -5, 7, -2, -18, 0, -1, 8, -3, 5, 0, -14, -2, 10, 0, 8, -7, -2, -2, ],
[-855, 51, -10, 3, -17, -6, 12, -9, -15, 0, 1, -5, -12, -4, -3, -11, -14, -5, -9, -3, -7, -2, 2, -1, 1, -2, -4, -3, 4, -4, 2, 0, -10, 1, 6, 3, 6, -4, -2, 2, ],
[-840, 32, -19, -1, -11, 1, 18, -5, -12, 6, -8, -22, -8, -11, -11, -12, -3, -5, -10, -1, 0, 0, -8, 4, -1, -5, -15, -1, -4, -1, 9, 5, -8, 0, -2, -13, -4, -4, 2, -5, ],
[-805, 46, -14, -6, -21, -6, 2, -5, -20, -21, -8, -24, -20, -10, -9, -17, -10, -10, -12, -9, -11, -9, -20, -3, -3, -8, -11, 2, -8, -1, 3, -3, -8, 2, 2, -7, -1, 0, 5, -2, ],
[-798, 47, -3, 2, -20, -6, -3, -9, -7, 11, -4, -22, -28, -12, -8, -16, -7, -4, -11, -10, -4, -1, -8, 1, 6, -1, -10, -7, 4, -7, -6, 1, -4, 6, 5, 0, -4, 1, 1, 2, ],
[-814, 50, -12, -11, -17, -3, 3, -13, -19, 1, -10, -19, -5, -3, 5, -9, -18, -2, -6, -5, -8, -4, -14, 4, 0, -10, 5, -3, 9, 0, -4, -2, -3, -2, 1, 0, -5, 1, 3, -2, ],
[-828, 49, -31, -23, -18, -9, -8, -15, -4, 8, -1, -17, -22, -3, -4, -15, 1, -4, -4, -2, -2, 1, -5, 5, 10, -4, 1, 3, 14, 0, -6, -5, -3, -1, 2, -2, 5, 6, 5, -1, ],
[-825, 50, -38, -8, -25, -10, -4, -16, -8, -5, -9, -13, -21, -8, -16, -3, 1, -3, -9, -5, 6, -4, -7, -2, -3, 4, 1, -1, 6, 7, 5, 2, -8, -5, 1, 1, -3, 0, -5, 3, ],
[-833, 27, -39, -15, -19, -17, -5, -18, -16, -20, -17, 3, -2, 11, 9, -11, -8, -12, -11, -16, -13, -18, -17, -9, -10, -15, -17, -2, 12, 8, -2, 0, -5, 5, 6, 0, 5, 4, 10, 3, ],
[-829, 47, -22, 13, -2, 1, 12, -12, -14, -8, -8, 3, -6, -4, -1, -6, -16, -6, -2, -10, -15, -26, -15, 1, 3, -8, 0, 9, 1, 4, 3, -4, -7, -3, -2, -1, 1, -3, -2, -3, ],
[-840, 37, -20, 0, -14, -4, 16, 0, 16, 4, -3, 1, -3, 6, 7, -4, -17, -14, -1, -21, -16, -22, -16, -10, 0, -13, -5, 2, 5, 10, 2, -1, 6, 8, 3, 4, -1, 0, 2, -4, ],
[-805, 63, 1, 4, -24, 9, 16, -5, -16, -13, -11, -9, -25, -10, -5, -10, -15, -7, -3, 4, -7, -9, 0, 8, 1, -11, -4, -13, 9, 7, -1, -7, -3, 5, 0, 2, -2, 0, -3, 0, ],
[-803, 68, -15, 4, -24, 3, -3, -13, -15, -21, -12, -9, -7, -15, -7, -14, -14, 6, 3, -7, -9, -6, 2, 2, 6, 0, -8, -3, 4, 3, 6, 1, -8, 0, 0, -2, 0, 3, 2, 4, ],
[-826, 38, -29, -3, -19, 10, 1, -9, -12, -16, 1, -8, -3, -19, -13, -14, -1, -2, -10, -11, -20, -4, -8, -2, -3, 2, -10, 1, -2, -7, -4, 5, -2, -5, -2, 5, 5, 3, 10, 0, ],
[-823, 51, -14, 5, -16, 4, -6, -13, -15, -17, -10, -6, -25, -20, -11, -9, -10, 1, -8, -7, -10, 1, -11, -10, -6, -4, -4, 2, -2, -2, -1, 3, 1, -1, -4, 0, -3, 0, -2, -5, ],
[-817, 46, -20, 0, -8, -12, -14, -12, -6, -2, -10, -8, -4, -12, 1, -6, -11, -15, -14, -5, -19, -13, -4, 1, 2, -10, -9, 1, -4, -5, 1, -3, -1, 2, 2, 3, 1, 1, 2, 0, ],
[-827, 45, -13, -6, -27, 4, -12, -24, -11, -11, -17, -4, -8, -19, -14, 0, -10, -19, -15, 2, -3, -8, -5, 7, 4, -12, -5, 7, 1, -1, 6, -8, -1, 0, -5, -4, 5, 6, -4, -4, ],
[-824, 33, -19, -3, -29, -4, 0, -19, -14, -14, -17, -8, -10, -16, -10, -3, -7, -20, -16, -6, -8, -6, -8, -7, 0, -8, 2, 11, 4, -2, 1, -4, -6, -2, -7, 1, -2, -1, -1, 4, ],
[-824, 32, -23, 0, -13, 1, 12, -8, -15, 0, -3, -4, -4, -13, -16, -16, -10, -8, -11, -18, -5, -7, 7, -4, 2, -5, 3, 2, 2, -2, 0, 4, 5, -1, -5, -4, -1, -5, 0, 2, ],
[-800, 53, -38, -8, -22, -3, 11, -2, -21, 5, -4, -9, -14, -6, -13, -12, -9, -6, 4, -10, 1, -7, -4, 6, 0, -7, -4, -5, 5, -6, 0, -3, -3, -1, -7, -1, -5, -5, -1, 1, ],
[-816, 46, -29, 2, -10, -3, 7, 3, -12, -13, -11, -24, -20, -18, -6, -9, -14, -6, -1, 2, -3, -5, -12, 11, -8, -9, -5, -6, 10, 6, 1, -1, -6, -5, 1, -3, -2, -3, 5, -1, ],
[-828, 39, -27, -7, -13, -12, -3, -10, -31, -30, -16, -5, -18, -20, -21, -9, -5, -7, -6, -6, -7, -13, -18, 0, 6, -18, -6, -13, -2, -4, -6, -8, -7, -6, -7, -2, 6, 0, 2, 5, ],
[-787, 62, -26, -19, -16, -13, 3, -16, -23, -11, -1, -8, -24, -12, 3, -14, -20, -12, -9, -15, -9, -8, 2, 7, 6, -12, 1, 2, -1, 3, 0, 1, -4, -6, -6, 2, -2, -10, -5, 6, ],
[-670, 105, -60, -57, -17, 7, 1, -43, -40, -15, 0, -2, -12, -9, -1, -14, -26, -18, -4, -1, 5, -2, 1, 0, -1, -14, -18, 6, 0, -2, 4, 1, -5, -2, -3, -2, -5, -2, 4, 2, ],
[-608, 119, -66, -78, -5, 17, -11, -53, -38, -5, 10, 20, 10, 3, 1, 1, -5, -19, -8, -7, 1, 3, 0, -7, -10, -5, -7, -1, 2, 12, 11, 2, -1, 3, -1, 5, 3, 5, 7, -2, ],
[-506, 131, -78, -91, -4, -7, -35, -38, -13, -9, 0, 14, 18, 1, 8, 14, -1, -1, -5, -8, 0, 7, -3, -6, -6, 1, -4, -7, 0, 1, -1, -1, 0, -2, -1, 0, -3, 0, -2, 1, ],
[-374, 152, -79, -89, 0, 2, -43, -23, -8, -19, -12, 8, 2, -16, -5, 7, -1, -1, 0, -9, -5, -3, -6, -2, -4, 0, 1, -3, 3, 4, 1, -5, 2, -3, -5, 2, -2, 4, 4, -2, ],
[-306, 130, -75, -95, 18, -13, -45, -24, -5, -15, -21, 14, 0, -11, -12, 13, 7, 3, -2, 3, -2, -5, 1, -9, -10, -16, -3, -6, -6, 1, 3, -10, -2, 8, -5, 0, -1, -2, -1, 0, ],
[-195, 134, -60, -121, 35, -24, -44, -28, -25, -9, -18, 19, -9, -8, -5, 2, 5, 1, 2, -9, 4, 3, -2, -9, -5, -4, 7, -2, -3, 8, 5, 3, 2, 1, 7, 1, -2, -1, 2, -2, ],
[-152, 100, -61, -125, 54, -32, -39, -22, -39, -14, -30, 3, -19, -21, -4, -3, 1, 1, 13, 5, 9, -1, -19, -20, -14, -10, 6, 2, -9, -4, 5, 2, -7, 0, 0, 1, -4, -9, -4, 1, ],
[-124, 84, -57, -91, 51, -41, -36, -26, -57, -6, -30, -7, -6, -9, -14, -8, -4, 3, 17, 10, 3, 9, 2, -13, -14, -9, 16, 9, 4, 3, -7, -2, 0, 0, -4, 5, 2, 1, -6, -4, ],
[-121, 88, -64, -88, 41, -51, -29, -28, -44, 0, -16, 5, -5, -13, -23, -14, -2, 9, 16, 14, 18, 13, 11, -11, -14, -13, 1, -11, 2, 10, -17, -11, 0, -2, -8, -1, -5, -1, -9, -10, ],
[-159, 103, -53, -81, 36, -48, -33, -29, -40, -6, -24, 5, 0, -21, -38, -8, 7, 3, 9, 15, 15, 18, 7, -20, -22, -12, 1, 0, 13, 4, -15, -10, 9, 3, -8, -4, -6, -3, -9, -10, ],
[-163, 82, -68, -92, 30, -51, -33, -32, -45, -9, -31, -1, -15, -36, -41, -16, 4, 2, 17, 19, 15, 18, 0, -21, -29, -21, 0, -6, 13, 5, -17, -8, 13, 3, -7, -6, -9, -11, -13, -16, ],
[-149, 92, -65, -94, 35, -45, -27, -26, -43, -13, -28, 6, -12, -35, -34, -6, 0, 5, 17, 15, 14, 15, 8, -20, -24, -16, 1, 9, 7, 0, -15, -7, 13, 2, -12, -3, -2, -7, -8, -12, ],
[-155, 102, -72, -91, 45, -46, -29, -20, -35, -12, -22, 4, -18, -33, -26, 1, 8, 4, 17, 23, 25, 14, 4, -23, -23, -14, -2, -1, 16, 1, -12, 0, 12, 4, -13, -10, -1, -6, -19, -14, ],
[-151, 112, -69, -85, 49, -44, -32, -19, -34, -8, -23, 4, -11, -32, -37, -3, 3, 4, 19, 21, 20, 16, -1, -23, -23, -9, 1, 3, 14, 1, -11, 1, 15, 5, -7, -3, 0, -7, -13, -18, ],
[-156, 112, -67, -77, 42, -49, -37, -20, -29, -6, -21, 8, -16, -27, -26, -1, 2, -2, 12, 19, 26, 13, -7, -16, -20, -5, -2, -6, 7, 2, -12, -6, 8, -3, -11, -2, 2, 0, -14, -18, ],
[-150, 106, -74, -94, 38, -49, -39, -23, -36, -9, -24, 2, -11, -32, -39, -7, -4, 1, 17, 14, 15, 19, 3, -19, -21, -12, -3, -6, 9, 7, -8, -6, 4, 1, -8, 0, -2, -5, -10, -20, ],
[-139, 107, -80, -90, 31, -50, -35, -32, -37, -1, -29, 2, -15, -27, -34, -15, -8, 1, 19, 10, 12, 19, 8, -14, -17, -11, -4, -6, 6, 17, 0, -9, 1, 4, -3, -2, 1, -3, -9, -18, ],
[-134, 101, -76, -81, 33, -41, -31, -25, -30, 4, -33, -5, -18, -16, -24, -17, -9, 3, 14, 11, 14, 14, 11, -9, -5, -9, -14, -11, -4, 17, 6, 3, -8, 10, 2, 1, -6, -2, -17, -21, ],
[-123, 105, -85, -79, 29, -35, -36, -11, -34, -3, -28, 1, -10, -16, -28, -20, -12, 3, 18, -1, 9, 19, 19, 2, -6, -18, -21, -8, 2, 17, 8, -7, -16, 18, 5, -2, -9, -7, -5, -9, ],
[-135, 108, -91, -83, 34, -37, -20, -16, -36, -6, -27, -2, -12, -21, -23, -18, -16, 2, 14, 0, 15, 19, 16, 12, -7, -24, -23, -13, 11, 16, 15, 2, -9, 12, 8, -6, -7, -10, -9, -4, ],
[-147, 102, -84, -89, 33, -38, -23, -12, -35, -10, -24, 1, -12, -16, -18, -18, -19, -4, 12, -3, 12, 14, 24, 22, 1, -24, -26, -15, 9, 15, 10, -1, -12, 11, 12, -2, -8, -9, -5, -2, ],
[-155, 98, -89, -91, 25, -42, -18, -12, -40, -7, -19, 3, -21, -23, -17, -28, -18, -7, 3, 2, 21, 10, 17, 18, -8, -30, -19, -12, 10, 26, 15, -14, -15, -12, -4, 6, 4, 5, 10, 11, ],
[-173, 88, -72, -78, 25, -31, 11, -11, -49, 0, -22, -8, -15, -10, -18, -19, -9, -13, -3, 0, 21, 15, 25, 29, 2, -28, -21, -12, 5, 22, 18, -5, -10, 2, 4, 5, 6, -5, 4, 6, ],
[-158, 89, -66, -69, 8, -48, 11, -20, -51, 14, -23, -9, -3, -9, -22, -16, -12, -13, 5, 4, 10, 15, 24, 15, -4, -27, -25, -16, 6, 20, 16, -8, -21, -2, -3, 3, -2, -3, 1, 5, ],
[-165, 97, -48, -67, -1, -44, 17, -29, -52, 15, -30, -10, -4, -11, -30, -20, -17, -17, 6, -7, 4, 15, 19, 9, -3, -27, -30, -12, 3, 22, 15, -6, -18, 0, 3, 0, -3, -3, 0, 4, ],
[-177, 85, -33, -76, -4, -35, 18, -29, -45, 15, -29, -1, 4, -7, -24, -17, -27, -9, 0, -5, 15, 17, 24, 15, -4, -28, -32, -18, -1, 18, 9, -9, -21, -7, -3, -1, -3, 0, 5, 7, ],
[-193, 104, -27, -73, -18, -47, 12, -33, -29, 1, -28, 4, -4, -3, -17, -28, -30, -3, -8, -5, 17, 9, 15, 24, -2, -22, -11, -15, 4, 25, 12, -7, -5, -5, 2, 6, -7, 4, 3, 13, ],
[-225, 112, -12, -81, -39, -55, -3, -30, -20, 8, -19, 8, -2, -5, -14, -31, -25, 2, -7, -10, 19, 8, -1, 21, 3, -21, -4, -12, -2, 9, 8, -14, -17, -6, -7, 2, -3, -3, 2, 14, ],
[-275, 146, 0, -48, -47, -54, -15, -35, -27, 0, -14, -6, -4, -5, -2, -17, -27, -7, 3, 2, 18, 2, 6, 14, -3, -19, -6, -2, 3, 18, 6, -10, -5, 3, -3, 3, -1, 0, 4, 2, ],
[-350, 123, -31, -34, -50, -51, 10, -26, -28, -1, -41, -11, -20, -29, -3, -9, -31, -10, 0, 10, 22, 9, 5, 17, -14, -35, -18, -10, -4, 9, -4, -7, -4, 4, -2, -9, -9, -10, -2, 0, ],
[-359, 141, -21, -31, -51, -35, 7, -25, -14, 3, -43, -13, -17, -23, -10, 2, -21, -12, 6, 17, 24, 7, 8, 16, -14, -24, -12, -3, 9, 12, 1, 0, 4, 14, 4, -5, -17, -6, 0, -6, ],
[-367, 143, -26, -49, -53, -33, -7, -25, -21, -10, -33, -26, -20, -18, -5, -6, -15, -5, 2, 1, 13, 7, 3, -2, -21, -24, -7, -7, -2, 2, -8, -14, 0, 6, -5, -7, -15, -3, -3, -5, ],
[-374, 157, -13, -54, -49, -32, 2, -19, -20, -10, -31, -22, -21, -27, -15, -8, -21, -3, 12, 5, 11, 9, -1, -4, -15, -17, -7, 5, 10, 12, -8, -11, -4, 4, -3, -6, -2, 8, 5, -2, ],
[-413, 175, -4, -50, -48, -23, -9, -28, -15, -4, -25, -24, -12, -8, -5, -12, -8, -2, 7, 7, 19, 13, 6, -11, -12, -5, -12, -4, 11, -4, -8, -9, -2, 3, 4, -10, -4, -3, 0, -9, ],
[-433, 169, -11, -54, -50, -39, -23, -43, -32, -10, -27, -45, -32, -15, -14, -26, -28, -25, -4, 10, 10, 3, 6, -13, -11, -7, -11, -7, 1, 5, -7, -7, -3, 6, -1, -4, 0, -1, -4, 4, ],
[-436, 169, -9, -52, -50, -27, -27, -45, -26, -19, -20, -30, -24, -7, -18, -22, -18, -14, 8, 9, 4, 10, 3, -4, -14, -14, -13, -14, 5, 4, -6, -15, -8, 2, 0, -9, -6, -2, -1, -7, ],
[-470, 157, 1, -36, -44, -21, -21, -37, -20, -16, -18, -29, -34, -23, -27, -24, -12, -2, 9, 15, 16, 11, -4, -2, -16, -23, -8, -3, 15, 16, 14, 4, 6, 4, -3, -4, -17, -16, -3, -14, ],
[-507, 147, 34, -14, -60, -34, -6, -29, -31, -18, -9, -5, -8, -21, -26, -25, -1, 3, 6, -1, 10, 8, -2, 2, 4, -11, -8, 1, 5, 8, 4, -4, -1, -1, -9, -7, -7, -10, -11, -14, ],
[-524, 147, 29, -19, -52, -31, -17, -32, -29, -32, -25, -22, -13, -16, -15, -15, -12, -18, -10, -10, -7, -6, 8, 1, -10, -11, -17, -8, -1, 7, 5, 5, -1, -3, -6, -4, -5, -8, -6, -10, ],
[-525, 138, 20, -15, -45, -24, -7, -27, -15, -26, -14, -8, -17, -11, -10, -24, -14, -9, -18, -13, 2, -2, 7, 11, 3, -6, -11, -2, 1, 1, -6, 3, 8, -1, -2, 0, -1, 2, 5, -6, ],
[-541, 123, 30, -7, -43, -41, -29, -22, -17, -4, -6, -10, -4, -5, -4, -23, -26, -14, -2, -1, -2, -3, -3, 12, 9, -12, -5, 0, 4, 1, -7, -4, 1, 7, 3, -8, -6, -1, 3, 8, ],
[-566, 98, 7, -3, -24, -19, -11, -27, -13, -9, -16, -1, -9, -7, -12, -10, -12, 2, 1, 2, 1, -1, -4, -2, 3, -6, -1, 9, 2, -5, -17, -5, -15, -7, -2, -5, 3, 4, 9, 2, ],
[-492, 45, 65, -61, 2, -36, -1, -23, -10, -6, -20, -8, -16, -12, -17, -18, -25, -3, -8, -5, 10, -4, 1, 3, 3, -2, -4, 1, -9, -2, -2, 2, 2, 2, -3, 1, 6, -6, -5, 0, ],
[-488, 33, 51, -79, -2, -43, -5, -21, -17, -6, -14, -5, -9, 0, -14, -19, -24, -4, -3, -1, 9, -2, 3, 2, 4, -7, -11, 4, -5, -1, -1, 1, -9, 1, -2, -5, 8, -2, -1, -2, ],
[-595, 32, 6, -37, -7, -32, 3, -11, -11, -14, -15, 6, -13, 1, -7, -29, -13, -3, -3, 3, 16, 5, 6, -2, -6, -10, -6, 3, -3, 2, -6, -3, -6, -13, -3, -1, -1, -2, 6, 0, ],
[-621, 21, 15, -33, -11, -30, 4, -24, -15, -8, -30, -15, -24, -4, 1, -17, -9, -11, -3, -9, -9, -12, 2, -5, -9, -16, -12, 0, -4, 2, 1, -6, -10, -12, -3, -1, 5, 1, 7, -5, ],
[-635, 39, 15, -21, -2, -28, 8, -23, -30, -8, -22, -17, -11, -6, -9, -19, -7, -5, 3, -5, -12, -12, -3, 1, 3, -9, -8, -5, -2, 5, -3, -5, -3, -1, 0, -6, -1, 2, 6, -2, ],
[-665, 34, -3, -16, -19, -18, -8, -23, -21, -10, -23, -9, -6, -6, -9, -11, -5, -3, -4, -12, -11, -10, -16, -8, -10, -4, -4, -10, -6, -3, -2, -12, 0, 5, -3, 5, -2, 1, 5, 1, ],
[-677, 31, -23, -19, -31, -17, -10, -21, -15, -3, -8, -19, -12, -6, -15, -12, -4, -1, -5, -18, -4, -6, -3, 12, -10, 1, -5, -3, 6, 4, 1, -8, -8, -2, 3, 4, -1, 7, 7, -5, ],
[-694, 22, -32, -28, -33, -20, -3, -21, -19, -3, 11, -5, -6, 0, 2, -10, -4, -10, -12, -28, -22, -15, -20, -11, -26, -17, -7, -9, -6, 1, 8, 1, 7, 6, 3, 0, 0, 6, 9, 0, ],
[-702, 34, -18, 8, -16, -2, -2, -24, -14, -19, 2, -13, -24, -9, -4, -14, -17, -1, -6, -14, -20, -6, -3, 11, -5, -11, -9, -9, 5, -1, -1, -8, -1, 1, 5, -1, -2, 0, 0, -4, ],
[-686, 41, -8, 17, -16, -1, 8, -13, -20, -12, -10, -10, -12, 0, 3, -15, -17, -5, -12, -9, -7, -8, -12, -2, -1, 2, -3, -8, -1, -4, 1, -4, 3, 3, 2, 3, -2, -2, 0, 1, ],
[-688, 51, -15, 7, -19, -26, 9, 4, -3, -12, -13, 1, -1, -9, 0, -19, -6, -5, -7, -5, 4, -4, -6, -4, 2, -2, -6, 1, 7, 8, -1, -4, -2, 9, 2, 4, -4, 0, -5, 4, ],
[-704, 53, -18, 1, -26, -12, 8, -6, 4, -1, -8, -8, -12, -13, -5, -23, -12, -6, -2, 0, -7, -4, -5, 11, 5, -11, -7, 6, 5, 2, 1, -9, -4, 7, 2, -4, -6, 5, 3, 7, ],
[-718, 44, -26, -4, -20, -10, -13, -14, 1, -2, -5, -4, -20, -2, -8, -20, -14, -2, 1, 5, -11, -4, -11, 0, -5, -7, -10, 0, 10, 8, -1, -1, -4, -3, 0, -2, -7, -5, -1, -7, ],
[-706, 58, -27, 9, -13, -13, -9, -23, -9, -19, -10, -2, -21, -9, -15, -17, -17, 1, 6, 16, 6, -11, -13, -5, -13, -11, -4, 1, -7, -2, 1, 3, -2, -3, 1, -3, -6, 0, -1, -2, ],
[-705, 69, -21, -9, -11, -21, -6, -13, -10, -19, -12, -6, -6, -17, 0, -3, -8, 2, 10, -1, 3, 2, -2, 4, -9, -15, -8, -1, -3, -1, 2, -4, -6, -7, -4, -1, -4, 2, 1, 1, ],
[-698, 63, -6, -11, -19, -20, -2, -3, -2, -12, -23, -1, -8, -2, 2, -4, -8, -8, 4, -14, -12, 0, -4, 0, 2, -2, -1, -7, 0, -6, 4, -3, -1, -3, 0, -2, -6, 0, 2, 1, ],
[-704, 49, -4, -6, -21, -22, -7, -9, -1, 6, -10, -9, -4, 6, 3, -3, -8, -7, 1, -13, -18, -2, 1, 12, 4, 0, 6, 5, -4, 6, 3, -2, -4, -2, -2, 2, 1, -3, -6, 0, ],
[-712, 55, -3, -2, -18, -11, 1, -17, -23, -15, -17, -9, -10, 11, -2, -9, -13, -12, -9, 4, -13, -8, 3, 11, 3, -7, -5, 0, -5, 6, 7, -3, -7, -7, -1, -3, -3, 2, 4, 0, ],
[-723, 37, -23, -6, -22, -19, -12, -24, -12, -19, -28, -17, -19, -6, -8, -9, 8, 7, -13, -9, -4, -5, -7, -2, -5, -12, -9, -5, -1, 2, 3, 0, -3, 0, -1, 2, -2, -6, 0, -5, ],
[-746, 24, -24, 5, -10, -2, -7, -23, -9, -13, -16, -11, -17, -3, 7, -7, -16, -10, -11, -10, -1, 11, 1, -2, 2, -12, -5, 3, 4, 1, -2, -1, -1, 1, 1, 4, -6, -6, 4, 0, ],
[-48, 81, 24, 16, 8, 4, 4, 0, 1, 0, 0, 0, 0, -1, -1, -1, -2, -2, 0, -2, -2, -2, -4, -2, -2, -1, -1, 1, 0, 0, -1, -1, -2, 1, 0, -1, -1, -1, -1, 0, ],
[180, 72, 12, 6, -3, -3, -7, -6, -5, -7, -5, -3, -5, -4, -4, -4, -5, -5, -3, -3, -3, -3, -4, -3, -4, -4, -2, -1, 0, -1, -1, -2, -3, -1, -1, -2, -2, -1, 0, -1, ],
[-12, 88, 30, 22, 14, 12, 7, 8, 5, 5, 6, 3, 2, 2, 3, 0, -2, -1, -1, -1, 0, -1, -4, -1, -1, -1, 1, 1, 0, 1, 1, -1, -1, 0, -1, 0, 0, 0, 0, 1, ],
])


In [8]:
plt.plot(tf_mfccs[0][55]*2)
plt.plot(input_prescale_gap[55] * FACTOR_SCALE)

IndexError: index 55 is out of bounds for axis 0 with size 49

# Network inference